<a href="https://colab.research.google.com/github/shackste/galaxy-generator/blob/separate_py_files/ACGAN_CVAE_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch implementation of combined Conditional Variable Auto Encoder (CVAE) and Auxiliary-Classifier Generative Adversarial Network (ACGAN)

continuation of work by Mohamad Dia

# Environment Setup

## Modules

In [ ]:
!pip install torchviz
!pip install wandb -qqq

In [ ]:
import sys 

import matplotlib.pyplot as plt
from torch import cat, add, ones, zeros

## include separate module files
#!git clone -b separate_py_files https://github.com/shackste/galaxy-generator.git
#sys.path.insert(0,"/content/galaxy-generator/python_modules/")

from google.colab import drive
drive.mount("/drive")

sys.path.insert(0,"/drive/MyDrive/FHNW/galaxy-generator/python_modules/")

from parameter import labels_dim, input_size, parameter
from file_system import root, folder_results
from helpful_functions import summarize, write_generated_galaxy_images_iteration
from sampler import make_training_sample_generator
from dataset import get_x_train, get_labels_train
from loss import loss_discriminator, loss_generator

import pipeline
from pipeline import VAE, VAEGAN
from discriminator import Discriminator4
from encoder import Encoder4
from decoder import Decoder4


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).

!!!!!!!!!!

galaxyzoo_data_cropped_nonnormalized.npy and training_solutions_rev1.csv must be placed in google drive under FHNW/galaxy_generator/
the results will be placed there, too.



# Track hyperparameter search via [wandb.ai](https://wandb.ai/shackste/galaxy-generator)

In [ ]:
import wandb
!wandb login



"""  USAGE
### the following has to be adjusted for every training run
### since hyperparameters can be changed on the fly, you find this in Training section

wandb.init(project="galaxy-generator", # top level identifier
           group="first", # second level identifier, to seperate several groups of tests
           job_type="training", # third level identifier, organize different jobs like training and evaluation
           tags=["first"], # temporary tags to organize different tasks together
           name="first", # bottom level identifier, label of graph in UI
           config=parameter.return_parameter_dict()  # here we fill the hyperparameters
)

## to follow evolution of loss or other measures wich epoch, after each iteration use:
wandb.log({"loss":loss})  ## here we usually pass loss and accuracy measures

## after training is done and all measures are written, finalize with
wandb.finish()
"""


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'  USAGE\n### the following has to be adjusted for every training run\n### since hyperparameters can be changed on the fly, you find this in Training section\n\nwandb.init(project="galaxy-generator", # top level identifier\n           group="first", # second level identifier, to seperate several groups of tests\n           job_type="training", # third level identifier, organize different jobs like training and evaluation\n           tags=["first"], # temporary tags to organize different tasks together\n           name="first", # bottom level identifier, label of graph in UI\n           config=parameter.return_parameter_dict()  # here we fill the hyperparameters\n)\n\n## to follow evolution of loss or other measures wich epoch, after each iteration use:\nwandb.log({"loss":loss})  ## here we usually pass loss and accuracy measures\n\n## after training is done and all measures are written, finalize with\nwandb.finish()\n'

# basic test Neural Networks
check whether the pipeline components work on their own

In [ ]:
summarize(Discriminator4)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]              64
         LeakyReLU-2           [-1, 16, 64, 64]               0
            Conv2d-3           [-1, 32, 32, 32]           4,640
       BatchNorm2d-4           [-1, 32, 32, 32]              64
         LeakyReLU-5           [-1, 32, 32, 32]               0
            Conv2d-6           [-1, 64, 16, 16]          18,496
       BatchNorm2d-7           [-1, 64, 16, 16]             128
         LeakyReLU-8           [-1, 64, 16, 16]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
        LeakyReLU-11            [-1, 128, 8, 8]               0
          Flatten-12                 [-1, 8192]               0
           Linear-13                 [-1, 1024]       8,389,632
        LeakyReL

In [ ]:
import numpy as np
from torch import rand


net = Encoder4()

print(np.sum([np.prod(p.size()) for p in net.parameters()]) == 8508656)
input_dummy = rand(3, *input_size)
label_dummy = rand(3, labels_dim)

net(input_dummy, label_dummy)

True


(tensor([[-0.3944,  1.0533, -0.0375,  0.5193, -0.6051, -0.3453,  0.4404,  0.1233],
         [-0.3326, -0.6303,  0.0375,  0.4422,  0.7960,  0.1599,  0.3348,  0.7549],
         [-0.4697,  0.3370,  0.2895, -0.3924, -0.3447,  0.0881,  0.1557, -0.9523]],
        grad_fn=<AddmmBackward>),
 tensor([[0.7459, 0.7544, 0.8106, 0.8917, 0.5913, 0.8075, 0.7445, 0.8992],
         [0.5824, 0.6460, 0.9327, 1.0385, 1.0698, 0.6082, 0.4591, 0.6607],
         [0.3900, 0.5706, 0.5481, 0.2879, 0.5479, 0.7917, 0.9547, 0.5178]],
        grad_fn=<SoftplusBackward>))

In [ ]:
net = Decoder4()

print(np.sum([np.prod(p.size()) for p in net.parameters()]))
input_dummy = rand(3, parameter.latent_dim)
label_dummy = rand(3, labels_dim)

net(input_dummy, label_dummy).shape

8524675


torch.Size([3, 3, 64, 64])

In [ ]:
net = VAE()
images_dummy = rand(5,3,64,64).cuda()
labels_dummy = rand(5,3).cuda()
pred = net(images_dummy, labels_dummy)
pred.shape

torch.Size([5, 3, 64, 64])

In [ ]:
net = VAEGAN()
input_dummy = rand(3,3,64,64).cuda()
labels_dummy = rand(3,3).cuda()
pred = net(input_dummy, labels_dummy)
pred.shape, pred[:,0]

(torch.Size([3, 1028]),
 tensor([0.4707, 0.4111, 0.4255], device='cuda:0', grad_fn=<SelectBackward>))

# Training Data
load data from files at google drive

In [ ]:
x_train = get_x_train()
labels_train = get_labels_train()
N_samples = x_train.shape[0]

# Training

In [ ]:
## start training with fresh, untrained networks
pipeline.decoder = Decoder4().cuda()
pipeline.encoder = Encoder4().cuda()
pipeline.discriminator = Discriminator4().cuda()

## if you want to change discriminator, encoder or decoder network:
#pipeline.decoder = YourDecoder().cuda()
## before you create the VAE and VAEGAN

## if you want to change any parameter:
parameter.alpha = 1.
parameter.beta = 1.
parameter.gamma = 1.
parameter.delta = 1.
parameter.zeta = 1.
## this can also be done during runtime

wandb.init(project="galaxy-generator", # top level identifier
           group="first", # second level identifier, to seperate several groups of tests
           job_type="training", # third level identifier, organize different jobs like training and evaluation
           tags=["first"], # temporary tags to organize different tasks together
           name="all one", # bottom level identifier, label of graph in UI
           config=parameter.return_parameter_dict()  # here we fill the hyperparameters
)




epochs = 20
batch_size = 128
steps = N_samples // batch_size
save_interval = 200

discriminator_losses = []
discriminator_losses_real = []
discriminator_losses_fake = []
generator_losses = []

valid = ones((batch_size,1)).cuda()
fake = zeros((batch_size,1)).cuda()

vae = VAE()
vaegan = VAEGAN()

iteration = 0
epoch = 0
step = 0





In [ ]:
def training_step():
    global iteration
    images, labels = next(sample_generator)

    # -------------------
    # Train Discriminator
    # -------------------
    vae.train(False)
    pipeline.discriminator.train(True)
    pipeline.discriminator.zero_grad()

    generated_images = vae(images, labels)
    target_real = cat((valid,labels), dim=1)
    prediction_real = pipeline.discriminator(images)[:,:1+labels_dim]
    target_fake = cat((fake, labels), dim=1)
    prediction_fake = pipeline.discriminator(generated_images)[:,:1+labels_dim]

    d_loss_real = loss_discriminator(target_real, prediction_real)
    d_loss_fake = loss_discriminator(target_fake, prediction_fake)
    d_loss = 0.5 * add(d_loss_fake, d_loss_real)
    discriminator_losses.append(d_loss)
    discriminator_losses_fake.append(d_loss_fake)
    discriminator_losses_real.append(d_loss_real)

    d_loss_real.backward()
    d_loss_fake.backward()
    pipeline.discriminator.optimizer.step()

    # ---------------
    # Train Generator
    # ---------------
    vae.train(True)
    pipeline.discriminator.train(False)
    pipeline.encoder.zero_grad()
    pipeline.decoder.zero_grad()

    generated_images = vae(images, labels)
    target = pipeline.discriminator(images)
    target[:,0] = 1
    target[:,1:1+labels_dim] = labels
    target = target.detach()
    prediction = pipeline.discriminator(generated_images)
    latent = pipeline.encoder(images, labels)

    g_loss = loss_generator(target, prediction, images, generated_images, latent)
    g_loss.backward()
    pipeline.encoder.optimizer.step()
    pipeline.decoder.optimizer.step()
    generator_losses.append(g_loss)

    ## save measures to wandb.ai
    wandb.log({"loss discriminator":d_loss, "loss generator":g_loss}) 


    iteration += 1

    print(f"iteration {iteration}, epoch {epoch+1}, batch {step+1}/{steps}," + \
          f"disc_loss {d_loss:.5}, (real {d_loss_real:.5}, fake {d_loss_fake:.5} ) gen_loss {g_loss:.5}")

    if not iteration % save_interval:
        write_generated_galaxy_images_iteration(iteration=iteration, images=generated_images.detach().cpu().numpy())


In [ ]:
while epoch < epochs:
    sample_generator = make_training_sample_generator(batch_size, x_train, labels_train)
    step = 0
    while step < steps:
        training_step()
        step += 1
    epoch += 1

    # save a plot of the costs
    plt.clf()
    plt.plot(discriminator_losses, label='discriminator cost')
    plt.plot(generator_losses, label='generator cost')
    plt.plot(discriminator_losses_fake, label='discriminator cost fake', linestyle=":")
    plt.plot(discriminator_losses_real, label='discriminator cost real', linestyle="-.")
    plt.yscale("log")
    plt.legend()
    plt.savefig(folder_results+"cost_vs_iteration.png")
    plt.close()


    ### really save?
    #pipeline.decoder.save()
    #pipeline.encoder.save()
    #pipeline.discriminator.save()

## after training is done and all measures are written, finalize with
wandb.finish()


iteration 1, epoch 1, batch 1/481,disc_loss 153.05, (real 152.05, fake 154.04 ) gen_loss 2809.7
iteration 2, epoch 1, batch 2/481,disc_loss 117.0, (real 118.37, fake 115.63 ) gen_loss 1700.5
iteration 3, epoch 1, batch 3/481,disc_loss 256.01, (real 272.59, fake 239.42 ) gen_loss 1257.7
iteration 4, epoch 1, batch 4/481,disc_loss 108.1, (real 112.93, fake 103.27 ) gen_loss 1093.3
iteration 5, epoch 1, batch 5/481,disc_loss 125.14, (real 126.7, fake 123.58 ) gen_loss 1013.6
iteration 6, epoch 1, batch 6/481,disc_loss 102.19, (real 109.5, fake 94.889 ) gen_loss 977.3
iteration 7, epoch 1, batch 7/481,disc_loss 95.428, (real 102.8, fake 88.056 ) gen_loss 930.57
iteration 8, epoch 1, batch 8/481,disc_loss 91.818, (real 98.888, fake 84.748 ) gen_loss 913.87
iteration 9, epoch 1, batch 9/481,disc_loss 90.455, (real 95.918, fake 84.993 ) gen_loss 891.67
iteration 10, epoch 1, batch 10/481,disc_loss 88.913, (real 96.969, fake 80.857 ) gen_loss 857.32
iteration 11, epoch 1, batch 11/481,disc_los

iteration 201, epoch 1, batch 201/481,disc_loss 83.296, (real 89.56, fake 77.032 ) gen_loss 486.14
iteration 202, epoch 1, batch 202/481,disc_loss 81.52, (real 85.707, fake 77.334 ) gen_loss 473.27
iteration 203, epoch 1, batch 203/481,disc_loss 80.975, (real 87.044, fake 74.905 ) gen_loss 475.36
iteration 204, epoch 1, batch 204/481,disc_loss 80.039, (real 85.084, fake 74.995 ) gen_loss 466.56
iteration 205, epoch 1, batch 205/481,disc_loss 79.264, (real 83.545, fake 74.984 ) gen_loss 475.18
iteration 206, epoch 1, batch 206/481,disc_loss 80.905, (real 86.729, fake 75.082 ) gen_loss 465.83
iteration 207, epoch 1, batch 207/481,disc_loss 81.387, (real 86.776, fake 75.998 ) gen_loss 467.82
iteration 208, epoch 1, batch 208/481,disc_loss 82.79, (real 88.802, fake 76.778 ) gen_loss 464.71
iteration 209, epoch 1, batch 209/481,disc_loss 78.811, (real 86.219, fake 71.403 ) gen_loss 467.31
iteration 210, epoch 1, batch 210/481,disc_loss 82.647, (real 88.703, fake 76.592 ) gen_loss 478.09
ite

iteration 401, epoch 1, batch 401/481,disc_loss 79.78, (real 86.321, fake 73.239 ) gen_loss 360.46
iteration 402, epoch 1, batch 402/481,disc_loss 79.437, (real 85.945, fake 72.929 ) gen_loss 362.63
iteration 403, epoch 1, batch 403/481,disc_loss 82.789, (real 88.295, fake 77.283 ) gen_loss 374.56
iteration 404, epoch 1, batch 404/481,disc_loss 80.671, (real 87.808, fake 73.534 ) gen_loss 362.02
iteration 405, epoch 1, batch 405/481,disc_loss 80.586, (real 86.947, fake 74.224 ) gen_loss 376.6
iteration 406, epoch 1, batch 406/481,disc_loss 81.307, (real 87.74, fake 74.875 ) gen_loss 374.87
iteration 407, epoch 1, batch 407/481,disc_loss 78.12, (real 85.038, fake 71.202 ) gen_loss 365.14
iteration 408, epoch 1, batch 408/481,disc_loss 81.166, (real 88.312, fake 74.02 ) gen_loss 356.62
iteration 409, epoch 1, batch 409/481,disc_loss 84.81, (real 91.47, fake 78.151 ) gen_loss 361.19
iteration 410, epoch 1, batch 410/481,disc_loss 79.543, (real 87.063, fake 72.024 ) gen_loss 362.74
iterati

iteration 601, epoch 2, batch 120/481,disc_loss 76.824, (real 84.093, fake 69.555 ) gen_loss 287.88
iteration 602, epoch 2, batch 121/481,disc_loss 84.135, (real 90.227, fake 78.042 ) gen_loss 286.79
iteration 603, epoch 2, batch 122/481,disc_loss 79.68, (real 84.827, fake 74.532 ) gen_loss 294.2
iteration 604, epoch 2, batch 123/481,disc_loss 74.356, (real 79.814, fake 68.898 ) gen_loss 281.2
iteration 605, epoch 2, batch 124/481,disc_loss 84.478, (real 91.219, fake 77.736 ) gen_loss 289.28
iteration 606, epoch 2, batch 125/481,disc_loss 82.362, (real 86.863, fake 77.86 ) gen_loss 289.96
iteration 607, epoch 2, batch 126/481,disc_loss 77.998, (real 83.164, fake 72.833 ) gen_loss 282.13
iteration 608, epoch 2, batch 127/481,disc_loss 75.519, (real 79.379, fake 71.659 ) gen_loss 291.64
iteration 609, epoch 2, batch 128/481,disc_loss 81.511, (real 87.679, fake 75.344 ) gen_loss 294.34
iteration 610, epoch 2, batch 129/481,disc_loss 84.065, (real 90.423, fake 77.708 ) gen_loss 294.52
iter

iteration 801, epoch 2, batch 320/481,disc_loss 80.768, (real 86.057, fake 75.479 ) gen_loss 237.66
iteration 802, epoch 2, batch 321/481,disc_loss 78.503, (real 83.245, fake 73.761 ) gen_loss 235.77
iteration 803, epoch 2, batch 322/481,disc_loss 75.843, (real 82.97, fake 68.715 ) gen_loss 228.89
iteration 804, epoch 2, batch 323/481,disc_loss 79.037, (real 84.245, fake 73.83 ) gen_loss 232.94
iteration 805, epoch 2, batch 324/481,disc_loss 79.174, (real 85.887, fake 72.462 ) gen_loss 237.95
iteration 806, epoch 2, batch 325/481,disc_loss 81.417, (real 88.04, fake 74.794 ) gen_loss 235.25
iteration 807, epoch 2, batch 326/481,disc_loss 76.361, (real 82.367, fake 70.355 ) gen_loss 233.79
iteration 808, epoch 2, batch 327/481,disc_loss 76.315, (real 81.004, fake 71.626 ) gen_loss 227.96
iteration 809, epoch 2, batch 328/481,disc_loss 79.373, (real 84.971, fake 73.776 ) gen_loss 235.17
iteration 810, epoch 2, batch 329/481,disc_loss 78.189, (real 83.712, fake 72.666 ) gen_loss 231.46
ite

...

iteration 9200, epoch 20, batch 61/481,disc_loss 77.564, (real 80.776, fake 74.351 ) gen_loss 179.02
iteration 9201, epoch 20, batch 62/481,disc_loss 76.585, (real 78.699, fake 74.471 ) gen_loss 175.66
iteration 9202, epoch 20, batch 63/481,disc_loss 77.966, (real 81.02, fake 74.911 ) gen_loss 184.91
iteration 9203, epoch 20, batch 64/481,disc_loss 79.655, (real 81.916, fake 77.394 ) gen_loss 185.24
iteration 9204, epoch 20, batch 65/481,disc_loss 78.647, (real 81.698, fake 75.596 ) gen_loss 187.39
iteration 9205, epoch 20, batch 66/481,disc_loss 73.523, (real 75.939, fake 71.108 ) gen_loss 179.18
iteration 9206, epoch 20, batch 67/481,disc_loss 76.628, (real 78.997, fake 74.259 ) gen_loss 178.0
iteration 9207, epoch 20, batch 68/481,disc_loss 71.834, (real 74.759, fake 68.908 ) gen_loss 176.86
iteration 9208, epoch 20, batch 69/481,disc_loss 74.188, (real 76.799, fake 71.577 ) gen_loss 176.33
iteration 9209, epoch 20, batch 70/481,disc_loss 76.909, (real 79.56, fake 74.257 ) gen_loss 

iteration 9400, epoch 20, batch 261/481,disc_loss 76.297, (real 79.02, fake 73.575 ) gen_loss 190.8
iteration 9401, epoch 20, batch 262/481,disc_loss 74.423, (real 77.134, fake 71.713 ) gen_loss 192.68
iteration 9402, epoch 20, batch 263/481,disc_loss 75.79, (real 78.426, fake 73.153 ) gen_loss 190.8
iteration 9403, epoch 20, batch 264/481,disc_loss 74.922, (real 77.75, fake 72.093 ) gen_loss 197.29
iteration 9404, epoch 20, batch 265/481,disc_loss 79.391, (real 82.315, fake 76.467 ) gen_loss 199.83
iteration 9405, epoch 20, batch 266/481,disc_loss 70.265, (real 72.507, fake 68.024 ) gen_loss 197.29
iteration 9406, epoch 20, batch 267/481,disc_loss 74.648, (real 77.539, fake 71.758 ) gen_loss 197.85
iteration 9407, epoch 20, batch 268/481,disc_loss 71.683, (real 74.405, fake 68.961 ) gen_loss 199.44
iteration 9408, epoch 20, batch 269/481,disc_loss 72.901, (real 75.63, fake 70.171 ) gen_loss 197.18
iteration 9409, epoch 20, batch 270/481,disc_loss 77.291, (real 80.038, fake 74.544 ) ge

iteration 9600, epoch 20, batch 461/481,disc_loss 76.767, (real 79.316, fake 74.218 ) gen_loss 213.91
iteration 9601, epoch 20, batch 462/481,disc_loss 79.03, (real 81.933, fake 76.128 ) gen_loss 216.99
iteration 9602, epoch 20, batch 463/481,disc_loss 76.609, (real 79.724, fake 73.495 ) gen_loss 208.29
iteration 9603, epoch 20, batch 464/481,disc_loss 75.495, (real 78.531, fake 72.459 ) gen_loss 213.1
iteration 9604, epoch 20, batch 465/481,disc_loss 75.77, (real 78.858, fake 72.682 ) gen_loss 213.51
iteration 9605, epoch 20, batch 466/481,disc_loss 76.544, (real 79.171, fake 73.917 ) gen_loss 209.01
iteration 9606, epoch 20, batch 467/481,disc_loss 77.187, (real 79.39, fake 74.984 ) gen_loss 206.65
iteration 9607, epoch 20, batch 468/481,disc_loss 71.455, (real 73.684, fake 69.226 ) gen_loss 196.28
iteration 9608, epoch 20, batch 469/481,disc_loss 74.021, (real 77.253, fake 70.789 ) gen_loss 192.61
iteration 9609, epoch 20, batch 470/481,disc_loss 75.769, (real 78.38, fake 73.158 ) g

loss discriminator,77.41829
loss generator,208.79089
_runtime,2402
_timestamp,1615460060
_step,9619


loss discriminator,▇█▆▄▆▄▄▃▆▃▄▅▂▃▅▃▇▄▆▃▂▄▄▄▄▃▃▃▄▁▃▃▃▄▄▃▃▃▄▄
loss generator,█▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


# Testing

In [ ]:
!pip install line_profiler
%load_ext line_profiler
%lprun -f training_step training_step()

In [ ]:
labels_train.dtype